# 03 · Backtest & Visualization

Validate the ML-driven signals using a vectorized long/short simulator and visualize equity curves.


**Goals**
- Load cached features, prices, and trained model
- Generate probability-driven signals and flip into positions
- Run a fast vectorized backtest with transaction costs
- Visualize performance metrics and sanity-check exposures


In [ ]:
from pathlib import Path

import joblib
import pandas as pd
import plotly.express as px

from momentum_lib import (
    bootstrap_env,
    pivot_ohlc,
    generate_signals,
    backtest_signals,
)

bootstrap_env(Path("../.env"))
data_dir = Path("../data")
features = pd.read_parquet(data_dir / "features.parquet")
prices = pivot_ohlc(pd.read_parquet(data_dir / "minute_bars.parquet"))
model = joblib.load(data_dir / "uprx_model.joblib")

overlap = features.index.intersection(prices.index)
X = features.loc[overlap]
prices = prices.loc[overlap]
print(f"Aligned frame shape: {X.shape}")


In [ ]:
signals = generate_signals(model, X, threshold=0.55)
results = backtest_signals(prices, signals)
results.head()


In [ ]:
results.attrs["stats"]


In [ ]:
fig = px.line(results.reset_index(), x="index", y="equity", title="Strategy Equity Curve")
fig.show()


In [ ]:
signals.value_counts(normalize=True).rename("signal_share")
